# Fetching crypto and tweets data.

In [1]:
from sklearn.preprocessing import OrdinalEncoder
from crypto_api import CryptoApi
import pandas as pd
import numpy as np
import re


#btc_df = pd.read_csv(r'Data\btc_data.csv',usecols=lambda x: x != "Unnamed: 0")

# Analysis of tweets

In [2]:
'''
TODO: GENERAL DATA TASKS:
0) find a way to deal with multiple tweets for a day
1) merge 2 datasets into 
2) imput missing data, maybe try interpolation or expectation maximization
    2.1) compare with mean, median imput methods
3) ivestigate relationship within data, maybe correlation matrix etc
'''

tweets_df = pd.read_csv(r'Data/elon_tweets.csv', index_col=0)
tweets_df['date'] = pd.to_datetime(tweets_df['date'])
                                                
tweets_df = (tweets_df
             .dropna(axis=1, how='all')
             .drop(['vibe','cashtags'], axis=1)) # 1 and 18 notna values respectively  

## Dealing with sparse columns

In [3]:
sparse_cols = tweets_df.columns[tweets_df.notnull().mean() < 1.0].values.copy()

mod_tweets_df = tweets_df.copy()
mod_tweets_df = (mod_tweets_df[mod_tweets_df['lang']=='en']
                 .drop(['id','url','source','sourceUrl'], axis=1)                 
                 .reset_index(drop=True)
                 .copy())

mod_tweets_df = mod_tweets_df.drop(['lang'], axis=1)

## Data cleaning and preprocessing

In [4]:
encoder = OrdinalEncoder()
mod_tweets_df['sourceLabel_encoded'] = encoder.fit_transform(mod_tweets_df['sourceLabel'].values.reshape(-1, 1))
mod_tweets_df['isReplied']   = [0 if type(tweet)==float else 1 for tweet in mod_tweets_df['inReplyToUser']]
mod_tweets_df['isMentioned'] = [0 if type(tweet)==float else 1 for tweet in mod_tweets_df['mentionedUsers']]

#mod_tweets_df = mod_tweets_df.drop(['sourceLabel','inReplyToUser','mentionedUsers'], axis=1)


def extract_dict(line: str, prepare_to_df: False):
    """Extracts data from a dict represented as string and makes it a dict.

    ## Parameters:
        line (str): row of a Series/DataFrame to be preprocessed.
        prepare_to_df (bool): prepares extracted dict to be wrapped into DataFrame.

    ## Returns:
        dict: extracted dict from string.
    """    

    extracted_content = dict(re.findall(r"'(\w+)': '?({.*}|datetime.datetime\(.*\)|[\w\d/:\. ]*)'?", line))
    
    # Wraps dict values into lists to be easily represented as a DataFrame row.
    if prepare_to_df:
        for key,value in extracted_content.items():
            if value == '':
                extracted_content[key] = [None]
            else:
                extracted_content[key] = value
        
    return extracted_content


new_df = mod_tweets_df.copy()     
extracted_df = (pd.DataFrame([*mod_tweets_df['user']
                              .apply(lambda x: extract_dict(x, True))])
                )
# TODO: try this json approach, but replace datetime via regex to str and then upcast to datetime again
# my_json = my_json.replace("'",'"').replace('None','null')
# json.loads(my_json)

new_df = (pd.concat([new_df, extracted_df], axis=1)
            .drop(['user','username','id','displayname','verified','created',
                    'location','protected','profileImageUrl','profileBannerUrl',
                    'rawDescription','renderedDescription','favouritesCount',
                    'friendsCount','mediaCount','statusesCount'], axis=1))

In [5]:
# Converting columns containing numbers to int after extraction.
for column in new_df:
    if 'Count' in column:
        new_df[column] = new_df[column].astype('Int64').copy()

In [6]:
new_df[['rawContent','isReplied','isMentioned']].query("rawContent.str.contains('@')")

new_df['mentionsCount'] = new_df['rawContent'].str.count(r'@[\w\d]+')
new_df['mentions'] = new_df['rawContent'].apply(lambda x : re.findall(r'(@[^\s]+)', x))

count = 0
for a,b in new_df[['mentionsCount','mentions']].values:
    if a==len(b):
        count +=1 
print(count==len(new_df))

new_df['charCount'] = new_df['rawContent'].apply(lambda x: len(x))
new_df = new_df.drop('descriptionLinks', axis=1)

True


In [7]:
#links = tweets_df[tweets_df.columns[tweets_df.columns.isin(new_df.columns)==False]]['links'].value_counts().copy()

tweets_df[tweets_df['rawContent']=='True']['quotedTweet'].iloc[0]

'{\'url\': \'https://twitter.com/ggreenwald/status/1625871270737809408\', \'date\': datetime.datetime(2023, 2, 15, 14, 54, 52, tzinfo=datetime.timezone.utc), \'rawContent\': "The corporate media\'s ability to -- overnight -- turn anyone who dissents in anyway into some sort of fascist or even Hitler-like figure, and then have millions of their followers go around mindlessly repeating it, is both impressive and chilling:", \'renderedContent\': "The corporate media\'s ability to -- overnight -- turn anyone who dissents in anyway into some sort of fascist or even Hitler-like figure, and then have millions of their followers go around mindlessly repeating it, is both impressive and chilling:", \'id\': 1625871270737809408, \'user\': {\'username\': \'ggreenwald\', \'id\': 16076032, \'displayname\': \'Glenn Greenwald\', \'rawDescription\': \'Journalist; Author; Host, @SystemUpdate_; Columnist, @Folha; Co-Founder: The Intercept, @TheInterceptBr; @abrigo_hope, @FreedomofPress, @ongcriadefavela.

In [240]:
import re
import json
import datetime


def clean_text(raw_text):    
    cleaned_text = re.sub(r' \'?(displayname|renderedDescription)\'?: (.*?)(\'|None),', '', raw_text)
    cleaned_text = (cleaned_text
                    .replace("'",'"')
                    .replace('None','null')
                    .replace('True','true')
                    .replace('False','false'))
    # cleaned_text = re.sub(r'(\w+)"(\w+)', r"\1'\2", cleaned_text)
    
    return cleaned_text


def deserialize(text):    
    deserialized_texts = []
    extract_dicts = re.findall(r'{.*?}',text)
    
    for str_dict in extract_dicts:
        cleaned_text = clean_text(str_dict)

        pattern = r'datetime.datetime\(.*\)'
        cleaned_text = re.sub(f'({pattern})',r'"\1"',cleaned_text)
        
        deserialized_text = json.loads(cleaned_text)
        
        if deserialized_text['created']!=None:
            deserialized_text['created'] = eval(deserialized_text['created'])
        
        deserialized_texts.append(deserialized_text)

    return deserialized_texts
    
    
new_df = new_df.drop('inReplyToUser',axis=1)
new_df['mentionedUsers'] = new_df['mentionedUsers'].apply(lambda x: deserialize(x) if type(x)==str else None)

# NLP Model Bulding

## Preprocessing and vectorizing

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, matutils, utils, models
import spacy


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    url_pattern = [{"label": "URL",
                    "pattern": [{"LIKE_URL": True}]}]

    ruler = nlp.add_pipe('entity_ruler', before='ner')
    ruler.add_patterns(url_pattern)
    
    texts_out = []
    if type(texts)!=list:
        texts = [texts]
    
    for text in texts:
        # TODO: consider using nlp.pipe which should be faster
        doc = nlp(text)
        cleaned_text = []
        for token in doc:
            if token.ent_type_ != 'URL' and not token.is_stop and token.pos_ in allowed_postags:
                cleaned_text.append(token.lemma_)
        final = ' '.join(cleaned_text)
        texts_out.append(final)

    return texts_out


def create_ngrams(texts):
    data_words = []
    for text in texts:
        new = utils.simple_preprocess(text)
        data_words.append(new)

    bigrams_phrases  = models.Phrases(data_words, min_count=3, threshold=50)
    trigrams_phrases = models.Phrases(bigrams_phrases[data_words], threshold=50)

    bigram  = models.phrases.Phraser(bigrams_phrases)
    trigram = models.phrases.Phraser(trigrams_phrases)

    data_bigrams = [bigram[doc] for doc in data_words]
    data_bigrams_trigrams = [trigram[bigram[doc]] for doc in data_bigrams]
    
    return data_bigrams_trigrams


def vectorize_texts(texts_ngrams):
    id2word = corpora.Dictionary(texts_ngrams)
    corpus = [id2word.doc2bow(text) for text in texts_ngrams]
    return id2word, corpus


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer


def make_custom_pipeline(steps):
    for i, step in enumerate(steps):
        steps.insert(i, (step[0], FunctionTransformer(step[1])))
        steps.remove(step)

    return Pipeline(steps)


# with open('lemmatized_texts.txt', 'r', encoding="utf-8") as f:
#     lemmatized_texts = f.readlines()
    
# lemmatized_texts = [line.replace('\n','') for line in lemmatized_texts]

# lemmatized_texts = lemmatization(new_df['rawContent'])
steps = [('lemmatization', lemmatization),
         ('trigrams', create_ngrams),
         ('vectorization', vectorize_texts)]

preprocessing_pipeline = make_custom_pipeline(steps)
id2word, corpus = preprocessing_pipeline.transform(new_df['rawContent'].values.tolist())

# data_bigrams_trigrams = create_ngrams(lemmatized_texts)
# id2word, corpus = vectorize_texts(data_bigrams_trigrams)

#tfidf = models.TfidfModel(corpus, id2word=id2word)
# tfidf_vectorizer = TfidfVectorizer(max_df=0.6,
#                                    min_df=5,
#                                    ngram_range=(1,3))
# tfidf_matrix = tfidf_vectorizer.fit_transform(lemmatized_texts)

# id2word = dict((v, k) for k, v in tfidf_vectorizer.vocabulary_.items())
# corpus = matutils.Sparse2Corpus(tfidf_matrix.T)

In [ ]:
# low_value = 0.03
# words = []
# words_missing_in_tfidf = []

# for i in range(0, len(corpus)):
#     bow = corpus[i]
#     low_value_words = []
#     tfidf_ids = [id for id,_ in tfidf[bow]]
#     bow_idf = [id for id,_ in bow]
#     low_value_words = [id for id, value in tfidf[bow] if value < low_value]
#     drops = low_value_words+words_missing_in_tfidf
    
#     for item in drops:
#         words.append(id2word[item])
    
#     # words with tfidf score of 0 will be missing
#     words_missing_in_tfidf = [id for id in bow_idf if id not in tfidf_ids]
#     new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
#     corpus[i] = new_bow

In [ ]:
from gensim.models import CoherenceModel


def bayesian_tuning(model, params_grid: dict, texts, verbose=False):
    # TODO: implement bayesian tuning
    models_scores = {}
    for i in range(95,170,5):
        lda_model = model(corpus=params_grid['corpus'],
                          num_topics=i,
                          id2word=params_grid['id2word'],
                          random_state=1,
                          passes=10,
                          per_word_topics=True)
        
        coherence_model_lda = CoherenceModel(model=lda_model, 
                                             texts=texts, 
                                             corpus=params_grid['corpus'], 
                                             dictionary=params_grid['id2word']
                                             )
        coherence_score = coherence_model_lda.get_coherence()
        
        models_scores.update({lda_model: coherence_score})
        
        if verbose:
            print(f'Topics {i:<3}: {coherence_score}')
    
    return models_scores


params_grid = {'corpus':corpus,  
               'num_topics':10, 
               'id2word':id2word, 
               'random_state':1, 
               'update_every':1, 
               'chunksize':3000, 
               'passes':2}

# lda_models_scores = bayesian_tuning(LdaMulticore, params_grid, texts, verbose=True)

In [ ]:
import pyLDAvis.gensim_models as gensimvis
from gensim.models import LdaMulticore
import pyLDAvis


pyLDAvis.enable_notebook()

lda_model = LdaMulticore(corpus=corpus,
                         num_topics=20,
                         id2word=id2word,
                         random_state=1,
                         passes=10,
                         per_word_topics=True)

vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

In [ ]:
def prepare_new_texts(texts: list[str]):
    steps = [('lemmatization', lemmatization),
            ('trigrams', create_ngrams)]

    corpus_pipeline = make_custom_pipeline(steps)
    texts_ngrams = corpus_pipeline.transform(texts)
    corpus = [id2word.doc2bow(text) for text in texts_ngrams]
    
    return corpus

In [ ]:
# corpus = [id2word.doc2bow(text) for text in texts_ngrams]
# lda_model.get_document_topics(corpus, minimum_probability=0)

In [ ]:
new_df['vectorized']  = preprocessing_pipeline.transform(new_df['rawContent'].values.tolist())[1]
new_df['TopicsProbs'] = new_df['vectorized'].apply(lambda x: dict(lda_model.get_document_topics(x, minimum_probability=0)))

In [ ]:
new_df['TopicsProbs']

In [ ]:
btc_df = pd.read_csv('Data/btc_data.csv')

In [ ]:
btc_df.iloc[0:1, 1:-2]

In [ ]:
btc_df

In [ ]:
datesCount = new_df.groupby(new_df['date'].dt.date)['date'].count()
new_df = new_df.merge(datesCount.rename('tweetsCount'), left_on=new_df['date'].dt.date, right_index=True, how='left')

In [ ]:
plot_this = new_df.groupby(new_df['date'].dt.date).count()['date']
plot_this

In [ ]:
#eval(text[13:-2])

# #text.replace('(null)','-(1)-')
# pattern = r'datetime.datetime\(.*\)'
# text = re.sub(f'({pattern})',r'"\1"',text)
# deserialized_text = json.loads(text)
# eval(re.findall(r'datetime.datetime\(.*\)', text)[0])
# #deserialized_text

import re


text = '{"username": "_andyoneal", "id": 19841555, "displayname": "Andy O"Neal", "rawDescription": null, "renderedDescription": null, "descriptionLinks": null, "verified": null, "created": null, "followersCount": null, "friendsCount": null, "statusesCount": null, "favouritesCount": null, "listedCount": null, "mediaCount": null, "location": null, "protected": null, "link": null, "profileImageUrl": null, "profileBannerUrl": null, "label": null}'

#text = re.sub(r'(\w+)"(\w+)', r"\1'\2", text)
#print(text)
json.loads(json.dumps(text))

In [ ]:
def drop_trash(text):
    use_cols = ['username','id','verified','created','followersCount',
                'friendsCount', 'statusesCount', 'favouritesCount',
                'listedCount', 'mediaCount', 'location', 'protected', 'label']
    
    i=0
    for word in text.split("'"):
        if word in use_cols:
            print(i, word)
            i+=1
    return text

text = "[{'username': 'Jason', 'id': 3840, 'displayname': '@jason', 'rawDescription': None, 'renderedDescription': None, 'descriptionLinks': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'link': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'label': None}, {'username': 'DeanPreston', 'id': 59243738, 'displayname': 'Dean Preston', 'rawDescription': None, 'renderedDescription': None, 'descriptionLinks': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'link': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'label': None}, {'username': 'GrowSF', 'id': 1299405799517634560, 'displayname': 'GrowSF', 'rawDescription': None, 'renderedDescription': None, 'descriptionLinks': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'link': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'label': None}]"
# re.sub(r"('displayname': )'(.*)' rawDescription", r'--VOID--', text)

# drop_trash(text)
#re.split(r"'(.*)':?", text)


In [ ]:
import json

total_anus = new_df.iloc[-2,14]

#re.findall(r'({.*})',total_anus)
json.loads(json.dumps(total_anus))